In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from datetime import datetime
import pandas as pd


current_date = pd.to_datetime(datetime.utcnow()).floor('H')
print(f'{current_date=}')

current_date=Timestamp('2025-06-28 14:00:00')


In [7]:
from src.inference import load_batch_of_features_from_store_v2

features = load_batch_of_features_from_store_v2(current_date)

2025-06-28 17:16:37,379 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-06-28 17:16:37,439 INFO: Initializing external client
2025-06-28 17:16:37,448 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-06-28 17:16:38,898 INFO: Python Engine initialized.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1235104


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (35.18s) 
[INFO] Rango de datos disponibles: 2024-01-01 00:00:00+00:00 a 2025-06-23 19:00:00+00:00
[INFO] La fecha pedida (2025-06-28 14:00:00+00:00) está fuera del rango disponible. Usando la última fecha disponible: 2025-06-23 19:00:00+00:00.


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (8.38s) 
[INFO] Horas de datos disponibles: 671
[INFO] Features requeridas: 672, Features que usaremos: 671
[WARNING] Faltan 1 horas de datos. Se rellenará con ceros.
[SUCCESS] Features generadas para 265 ubicaciones con 671/672 horas de datos.


In [9]:
from src.inference import load_model_from_registry, get_model_predictions


model = load_model_from_registry()
predictions = get_model_predictions(model, features)

2025-06-28 17:24:37,120 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-06-28 17:24:37,172 INFO: Initializing external client
2025-06-28 17:24:37,179 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-06-28 17:24:38,724 INFO: Python Engine initialized.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1235104


Downloading: 0.000%|          | 0/836918 elapsed<00:00 remaining<?

[LightGBM] [Warning] feature_fraction is set=0.9230661239064173, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9230661239064173
[LightGBM] [Warning] bagging_fraction is set=0.6687867505273777, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6687867505273777


In [19]:
# Ajustar el DataFrame para el feature group de predicciones
predictions['pickup_ts'] = current_date  # Usar pickup_ts en lugar de pickup_hour
predictions.rename(columns={'predicted_demand': 'predictions'}, inplace=True)  # Renombrar para consistencia
predictions

,pickup_location_id,predictions,pickup_hour,pickup_ts
0,1,1.0,2025-06-28 14:00:00,2025-06-28 14:00:00
1,2,0.0,2025-06-28 14:00:00,2025-06-28 14:00:00
2,3,0.0,2025-06-28 14:00:00,2025-06-28 14:00:00
3,4,3.0,2025-06-28 14:00:00,2025-06-28 14:00:00
4,5,0.0,2025-06-28 14:00:00,2025-06-28 14:00:00
...,...,...,...,...
260,261,41.0,2025-06-28 14:00:00,2025-06-28 14:00:00
261,262,57.0,2025-06-28 14:00:00,2025-06-28 14:00:00
262,263,91.0,2025-06-28 14:00:00,2025-06-28 14:00:00
263,264,21.0,2025-06-28 14:00:00,2025-06-28 14:00:00


In [20]:
from src.feature_store_api import get_or_create_feature_group
import src.config as config

# Usar el feature group correcto para predicciones
feature_group = get_or_create_feature_group(config.FEATURE_GROUP_PREDICTIONS_METADATA)

2025-06-28 18:23:10,257 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-06-28 18:23:10,294 INFO: Initializing external client
2025-06-28 18:23:10,298 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-06-28 18:23:11,676 INFO: Python Engine initialized.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1235104


In [21]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1235104/fs/1218641/fg/1491584


Uploading Dataframe: 100.00% |██████████| Rows 265/265 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: model_predictions_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1235104/jobs/named/model_predictions_feature_group_1_offline_fg_materialization/executions


(Job('model_predictions_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)